<a href="https://colab.research.google.com/github/Thomascc123/Proyecto-Kaggle-Modelos/blob/master/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import math
import unicodedata

# Cargando los archivos

In [4]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.36GB/s]


In [5]:
!unzip udea*.zip > /dev/null

# PROCESING TRAIN

## Funciones para procesar datos

In [23]:
def deleteColumns(df, columns):
    """
    Elimina las columnas especificadas de un DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columns (list): Lista de nombres de columnas a eliminar.

    Returns:
        None: elimina las co
    """

    df.drop(columns, axis=1, inplace=True)

In [6]:
def fillDefault(df, column, value):
    """
    Rellena los valores nulos de una columna en un DataFrame con un valor por defecto.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a modificar.
        value (any): Valor con el que se reemplazarán los nulos.

    Returns:
        None: Modifica el DataFrame directamente.
    """
    df[column] = df[column].fillna(value)

In [24]:
def fillColumnsDefaults(df, columnsDefaults):
  """
    Rellena los valores nulos de multiples columnas en un DataFrame con valores valor
    por defecto de cada columna.

     Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columnsDefaults (str, str): Diccionario que contiene como clave las columnas
        a modificar y como valor el valor con el que se reemplazarán los nulos.

    Returns:
        None: Modifica el DataFrame directamente.

  """
  for column in columnsDefaults():
    value = columnsDefaults[column]
    fillDefault(df, column, value)

In [7]:
def replaceNullByList(df, column, values):
    """
    Reemplaza los valores nulos de una columna en un DataFrame con valores aleatorios de una lista dada.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a modificar.
        values (list): Lista de valores posibles para reemplazar los nulos.

    Returns:
        None: Modifica el DataFrame directamente.
    """
    numNulls = df[column].isna().sum()
    randomReplaces = np.random.choice(values, size=numNulls, replace=True)
    df.loc[df[column].isna(), column] = randomReplaces

In [25]:
def replacesColumnsNullsByLists(df, columnsLists):
    """
    Reemplaza los valores nulos de varias columna especificadas de un DataFrame con valores aleatorios
    de listas dadas para cada columna.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columnsLists (str, list): Dicionario con clave las columnas a modificar y como valor
        una lista de valores posibles para reemplazar los nulos.

    Returns:
        None: Modifica el DataFrame directamente.
    """

    for column in columnsLists():
        values = columnsLists[column]
        replaceNullByList(df, column, values)

In [8]:
def getDistincValues(df, column):
    """
    Obtiene una lista de valores únicos no nulos de una columna en un DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a analizar.

    Returns:
        list: Lista de valores distintos sin incluir nulos.
    """
    return df[column].dropna().unique().tolist()

In [31]:
def replaceNullByColumnsValues(df, column):
    """
    Reemplaza los valores nulos aleatoriamente por valores no nulos de la columna.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a modificar.

    Returns:
        None: Modifica el DataFrame directamente.
    """

    values = getDistincValues(df, column)
    numNulls = df[column].isna().sum()
    randomReplaces = np.random.choice(values, size=numNulls, replace=True)
    df.loc[df[column].isna(), column] = randomReplaces

In [32]:
def replacesNullsColumnsByColumnsValues(df, columns):
    """
    Reemplaza los valores nulos de las columnas aleatoriamente por valores no nulos
    de las columnas ingresadas.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columns (list): lista con los nombres de las columnas a modificar.

    Returns:
        None: Modifica el DataFrame directamente.
    """
    for column in columns:
        replaceNullByColumnsValues(df, column)

In [9]:
def onehotEncode(df, column):
    """
    Aplica codificación one-hot manual a una columna categórica de un DataFrame.

    Crea nuevas columnas binarias (0 o 1) por cada valor único en la columna original,
    y luego elimina la columna original del DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a codificar.

    Returns:
        pandas.DataFrame: DataFrame con las nuevas columnas codificadas y sin la columna original.
    """
    values = df[column].unique().tolist()

    for value in values:
        df[column + '_' + value] = (df[column] == value).astype(int)

    df.drop(columns=[column], inplace=True)
    return df

In [26]:
def onehotEncodeColumns(df, columns):
    """
      Aplica codificación one-hot manual a las columnas ingresadas de un DataFrame.

      Crea nuevas columnas binarias (0 o 1) por cada valor único en la columna original,
      y luego elimina la columna original del DataFrame.

      Args:
          df (pandas.DataFrame): DataFrame que contiene los datos.
          columns (list): Nombre de las columnas a codificar.

      Returns:
          pandas.DataFrame: DataFrame con las nuevas columnas codificadas y sin la columna original.
    """
    for column in columns:
        df = onehotEncode(df, column)
    return df

In [27]:
def formatText(df, column):
    """
    Elimina los acentos de los valores de texto en una columna de un DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a formatear.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """
    def removeAccents(text):
        if isinstance(text, str):
            return ''.join(
                c for c in unicodedata.normalize('NFKD', text)
                if not unicodedata.combining(c)
            )
        return text

    df[column] = df[column].apply(removeAccents)

In [12]:
def valuesPercentage(df, column):
    """
    Calcula el porcentaje de frecuencia de cada valor en una columna de un DataFrame.

    Aplica la función `formatText` para eliminar acentos antes del cálculo.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a analizar.

    Returns:
        pandas.Series: Serie con los porcentajes de aparición de cada valor.
    """
    df = df.copy()
    formatText(df, column)
    percentages = (df[column].value_counts(normalize=True) * 100).round(2)
    return percentages

In [13]:
def valuesFilterSignificance(df, column, min_percentage, printInfo=True):
    """
    Filtra los valores de una columna según un porcentaje mínimo de aparición.

    Usa `valuesPercentage` para calcular la frecuencia relativa de cada valor y
    conserva solo aquellos que superen el umbral especificado.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a analizar.
        min_percentage (float): Porcentaje mínimo requerido para conservar un valor.
        printInfo (bool, opcional): Si es True, muestra información sobre los valores filtrados.
            Por defecto True.

    Returns:
        pandas.Series: Serie con los valores y sus porcentajes que cumplen el criterio.
    """
    percentages = valuesPercentage(df, column)
    filtered = percentages[percentages >= min_percentage]

    if printInfo:
        print(filtered)
        print("Porcentaje acumulado: ", filtered.sum())
        print("Número de valores: ", filtered.shape[0])

    return filtered

In [14]:
def groupValuesByPercentage(df, column, min_percentage, valueGroup='Otros'):
    """
    Agrupa en una misma categoría los valores de una columna cuyo porcentaje de aparición
    sea menor al umbral especificado.

    Utiliza `valuesFilterSignificance` para identificar los valores significativos
    y reemplaza los menos frecuentes por una etiqueta genérica (por defecto 'Otros').

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a procesar.
        min_percentage (float): Porcentaje mínimo para considerar un valor como significativo.
        valueGroup (str, opcional): Nombre de la categoría donde se agruparán los valores poco frecuentes.
            Por defecto 'Otros'.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """
    formatText(df, column)
    filtered = valuesFilterSignificance(df, column, min_percentage, printInfo=False)
    values_to_keep = filtered.index.tolist()
    df.loc[~df[column].isin(values_to_keep), column] = valueGroup

In [15]:
def groupValuesByList(df, column, valuesToGroup, valueGroup='Otros'):
    """
    Agrupa en una misma categoría los valores de una columna que pertenezcan a una lista dada.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a modificar.
        valuesToGroup (list): Lista de valores que serán reemplazados por la categoría genérica.
        valueGroup (str, opcional): Nombre de la categoría donde se agruparán los valores especificados.
            Por defecto 'Otros'.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """
    df.loc[df[column].isin(valuesToGroup), column] = valueGroup

In [34]:
def groupsValuesColumnsByLists(df, columnsValuesToGroup):
    """
    Agrupa en una misma categoría los valores de las columnas dadas
    que pertenezcan a las respectivas listas dadas.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columnsValuesToGroup (str, dict): Diccionario que tiene como clave el nombre de la columna a modificar
        y como valor una lista con los valores que serán reemplazados por la categoría genérica.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """
    for column in columnsValuesToGroup:
        valuesToGroup = columnsValuesToGroup[column]
        groupValuesByList(df, column, valuesToGroup)

In [18]:
def replaceBinaryValues(oneValue, zeroValue, df, column):
    """
    Reemplaza valores de una columna por equivalentes binarios (1 y 0).

    Args:
        oneValue (any): Valor que se reemplazará por 1.
        zeroValue (any): Valor que se reemplazará por 0.
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a modificar.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """
    df[column] = df[column].replace({oneValue: 1, zeroValue: 0})
    df[column] = df[column].astype(int)

In [28]:
def replaceBinaryValuesColumns(df, columnsValues):
    """
    Reemplaza valores de una columna por equivalentes binarios (1 y 0).

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columnsValues (str, tuple): Diccionario que tiene como clave el nombre de la columna a modificar
        y como valor una tupla con los valores que se reemplazarán por 1 y 0 respectivamente.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """

    for column in columnsValues:
        oneValue = columnsValues[column][0]
        zeroValue = columnsValues[column][1]
        replaceBinaryValues(oneValue, zeroValue, df, column)

In [19]:
def replaceDiscreteValues(replaceDict, df, column):
    """
    Reemplaza valores discretos en una columna según un diccionario de mapeo.

    Args:
        replaceDict (dict): Diccionario que define los reemplazos.
            Ejemplo: {'A': 'Alto', 'B': 'Bajo'}.
        df (pandas.DataFrame): DataFrame que contiene los datos.
        column (str): Nombre de la columna a modificar.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """
    df[column] = df[column].replace(replaceDict)

In [29]:
def replaceDiscreteValuesColumns(df, columnsValues):
    """
    Reemplaza valores discretos en una columna según un diccionario de mapeo.

    Args:
        df (pandas.DataFrame): DataFrame que contiene los datos.
        columnsValues (str, dict): Diccionario que tiene como clave el nombre de la columna a modificar
        y como valor un diccionario con los valores que se reemplazarán por sus equivalentes.

        Ejemplo del diccionario de valores: {'A': 'Alto', 'B': 'Bajo'}.

    Returns:
        None: Modifica directamente la columna del DataFrame.
    """

    for column in columnsValues:
        replaceDict = columnsValues[column]
        replaceDiscreteValues(replaceDict, df, column)

In [20]:
traindf = pd.read_csv("train.csv")
traindf.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [21]:
traindf.columns

Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3', 'INDICADOR_4'],
      dtype='object')

In [36]:
def procesDataFrame(df, deleteColumns, columnsDefaults, columnsNullsList, columnsNullsValues,
                    relativeFrecuencytoGrouping, columnsListsReplacesValues, onehotColumns,
                    columnsBinaryValues, discreteColumnsValues):
  ###################### Eliminando columnas ######################

  #Eliminando columna duplicada
  deleteColumns(df, 'F_TIENEINTERNET.1')

  #Eliminando columnas irrelevantes
  deleteColumns(df, deleteColumns)

  ###################### Eliminando nulos ######################

  #Cambiando nulos a valores por defecto
  fillColumnsDefaults(df, columnsDefaults)

  #cambiando nulos por valores de una lista
  replacesColumnsNullsByLists(df, columnsNullsList)

  #cambiando nulos por valores de las columnas
  replacesNullsColumnsByColumnsValues(df, columnsNullsValues)

  ###################### Conversión a valores numericos ######################

  #Arreglando la columna de programas academicos
  groupValuesByPercentage(df, 'E_PRGM_ACADEMICO', relativeFrecuencytoGrouping, 'OTROS')
  df = onehotEncode(df, 'E_PRGM_ACADEMICO')

  #Agrupando valores similares de columnas
  groupsValuesColumnsByLists(df, columnsListsReplacesValues)

  #One hot encoding
  df = onehotEncodeColumns(df, onehotColumns)

  #Convirtiendo valores binarios
  replaceBinaryValuesColumns(df, columnsBinaryValues)

  #Reemplazar valores discretos con orden
  replaceDiscreteValuesColumns(df, discreteColumnsValues)

  return df

In [ ]:
copytrain = traindf.copy()

procesDataFrame(
    copytrain,
    deleteColumns = ['E_PRIVADO_LIBERTAD'],
    columnsDefaults = {
      'E_VALORMATRICULAUNIVERSIDAD': "No pagó matrícula",
      'E_HORASSEMANATRABAJA': '0',
      'F_TIENEINTERNET': 'No',
      'F_TIENELAVADORA': 'No',
      'E_PAGOMATRICULAPROPIO': "No",
      'F_TIENECOMPUTADOR': "No"
    },
    columnsNullsList = {

    },
    columnsNullsValues = ['F_ESTRATOVIVIENDA', 'F_EDUCACIONPADRE', 'F_TIENEAUTOMOVIL', 'F_EDUCACIONMADRE'],

   )
